# MCP Server Discovery

Automatically discover and connect to MCP servers from your configuration.

**Features:**
- Auto-discover servers from Claude Code config
- List available tools from each server
- Connect and call tools across servers

## Understanding MCP Discovery

MCP servers can be discovered from:
- `~/.config/Claude/claude_desktop_config.json` (Claude Code)
- `~/.agentcards/config.yaml` (AgentCards)
- Custom config files

In [ ]:
import {
  MCPServerDiscovery,
  MCPClient,
  type MCPServer,
  type MCPTool
} from "jsr:@casys/mcp-gateway";

console.log("✅ MCP Discovery module loaded");

## Example: MCP Config Structure

This is what a typical MCP config looks like:

In [ ]:
const exampleConfig = {
  "mcpServers": {
    "filesystem": {
      "command": "npx",
      "args": ["-y", "@modelcontextprotocol/server-filesystem", "/tmp"]
    },
    "brave-search": {
      "command": "npx",
      "args": ["-y", "@anthropic/mcp-server-brave-search"],
      "env": {
        "BRAVE_API_KEY": "your-api-key"
      }
    },
    "casys-gateway": {
      "command": "deno",
      "args": ["run", "--allow-all", "jsr:@casys/mcp-gateway"]
    }
  }
};

console.log("📋 Example MCP Config:");
console.log(JSON.stringify(exampleConfig, null, 2));

## Simulating Server Discovery

Since we're in a notebook, we'll simulate what discovery looks like:

In [ ]:
// Simulated discovered servers
const discoveredServers: MCPServer[] = [
  {
    name: "filesystem",
    command: "npx",
    args: ["-y", "@modelcontextprotocol/server-filesystem", "/tmp"],
    status: "available"
  },
  {
    name: "casys-sandbox",
    command: "deno",
    args: ["run", "--allow-all", "jsr:@casys/mcp-gateway"],
    status: "available"
  },
  {
    name: "database",
    command: "node",
    args: ["db-mcp-server.js"],
    status: "unavailable"
  }
];

console.log("🔍 Discovered MCP Servers:\n");
for (const server of discoveredServers) {
  const status = server.status === "available" ? "✅" : "❌";
  console.log(`${status} ${server.name}`);
  console.log(`   Command: ${server.command} ${server.args?.join(' ')}`);
  console.log();
}

## Tools from Each Server

Each MCP server exposes different tools:

In [ ]:
// Simulated tools from servers
const serverTools: Record<string, MCPTool[]> = {
  "filesystem": [
    { name: "read_file", description: "Read contents of a file", inputSchema: {} },
    { name: "write_file", description: "Write contents to a file", inputSchema: {} },
    { name: "list_directory", description: "List files in a directory", inputSchema: {} },
  ],
  "casys-sandbox": [
    { name: "execute_code", description: "Execute TypeScript/JavaScript safely", inputSchema: {} },
    { name: "execute_dag", description: "Execute a DAG workflow", inputSchema: {} },
    { name: "search_tools", description: "Semantic search for relevant tools", inputSchema: {} },
  ]
};

console.log("🔧 Tools by Server:\n");
for (const [server, tools] of Object.entries(serverTools)) {
  console.log(`📦 ${server}:`);
  for (const tool of tools) {
    console.log(`   • ${tool.name}: ${tool.description}`);
  }
  console.log();
}

## Gateway Aggregation

The MCP Gateway aggregates all tools into a single interface:

In [ ]:
// Flatten all tools with server prefix
const allTools = Object.entries(serverTools).flatMap(([server, tools]) =>
  tools.map(tool => ({
    ...tool,
    fullName: `${server}:${tool.name}`,
    server
  }))
);

console.log("🌐 Gateway: All Available Tools\n");
console.log(`Total: ${allTools.length} tools from ${Object.keys(serverTools).length} servers\n`);

for (const tool of allTools) {
  console.log(`${tool.fullName}`);
  console.log(`   ${tool.description}`);
  console.log();
}

## Semantic Tool Search

Find relevant tools using natural language:

In [ ]:
// Simple keyword-based search (real version uses embeddings)
function searchTools(query: string, tools: typeof allTools) {
  const keywords = query.toLowerCase().split(' ');
  return tools
    .map(tool => {
      const text = `${tool.name} ${tool.description}`.toLowerCase();
      const score = keywords.filter(kw => text.includes(kw)).length;
      return { ...tool, score };
    })
    .filter(t => t.score > 0)
    .sort((a, b) => b.score - a.score);
}

const queries = [
  "execute code safely",
  "read file contents",
  "workflow dag"
];

console.log("🔍 Semantic Tool Search:\n");
for (const query of queries) {
  console.log(`Query: "${query}"`);
  const results = searchTools(query, allTools);
  for (const tool of results.slice(0, 2)) {
    console.log(`   → ${tool.fullName} (score: ${tool.score})`);
  }
  console.log();
}

## Summary

**MCP Discovery enables:**
- ✅ Auto-detect servers from config files
- ✅ List tools from all connected servers
- ✅ Aggregate tools into single gateway
- ✅ Semantic search across all tools

**The Gateway Pattern:**
```
                    ┌─ Server A (filesystem)
Client → Gateway ───┼─ Server B (sandbox)
                    └─ Server C (database)
```

LLMs interact with ONE gateway that routes to ALL servers!